In [1]:
# Data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Bokeh libraries
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
from bokeh.models import ColumnDataSource, HoverTool, CategoricalColorMapper, LinearColorMapper, ColorBar, Slider
from bokeh.palettes import Spectral5, Category20b, Viridis, Cividis, Viridis256
from bokeh.transform import factor_cmap, transform

In [3]:
# The figure will be rendered inline a my Jupyter Notebook
output_notebook()

# Use output_file('dash.html') to open in the browser

Loading BokehJS ...

In [4]:
# Reading data
df = pd.read_csv('../data/dash_sample_weekly_2019.csv')
df.head()

,Player,Pos,Team,PassingYds,PassingTD,Int,PassingAtt,Cmp,RushingAtt,RushingYds,...,Rec,Tgt,ReceivingYds,ReceivingTD,FL,PPRFantasyPoints,StandardFantasyPoints,HalfPPRFantasyPoints,Week,Year
0,Lamar Jackson,QB,BAL,324.0,5.0,0.0,20.0,17.0,3.0,6.0,...,0.0,0.0,0.0,0.0,0.0,33.56,33.56,33.56,1,2019
1,Dak Prescott,QB,DAL,405.0,4.0,0.0,32.0,25.0,4.0,12.0,...,0.0,0.0,0.0,0.0,0.0,33.40,33.40,33.40,1,2019
2,Robert Griffin,QB,BAL,55.0,1.0,0.0,6.0,6.0,4.0,9.0,...,0.0,0.0,0.0,0.0,0.0,7.10,7.10,7.10,1,2019
3,Patrick Mahomes,QB,KAN,378.0,3.0,0.0,33.0,25.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,27.32,27.32,27.32,1,2019
4,Kirk Cousins,QB,MIN,98.0,1.0,0.0,10.0,8.0,6.0,4.0,...,0.0,0.0,0.0,0.0,0.0,14.32,14.32,14.32,1,2019


In [5]:
# Removing NaN 
df = df.dropna()

In [6]:
df.describe()

,PassingYds,PassingTD,Int,PassingAtt,Cmp,RushingAtt,RushingYds,RushingTD,Rec,Tgt,ReceivingYds,ReceivingTD,FL,PPRFantasyPoints,StandardFantasyPoints,HalfPPRFantasyPoints,Week,Year
count,4779.000000,4779.000000,4779.000000,4779.000000,4779.000000,4779.000000,4779.000000,4779.000000,4779.000000,4779.000000,4779.000000,4779.000000,4779.000000,4779.000000,4779.000000,4779.000000,4779.000000,4779.0
mean,26.974472,0.166771,0.085792,3.735719,2.370998,2.801214,12.093116,0.093534,2.370998,3.467880,26.973635,0.166771,0.062984,9.288018,6.917020,8.102519,9.071354,2019.0
std,80.650203,0.616856,0.396939,10.933560,6.993437,5.254732,25.969117,0.349966,2.357767,3.370824,32.372313,0.433063,0.261224,8.185389,7.235407,7.634533,5.021275,0.0
min,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-10.000000,0.000000,0.000000,0.000000,-9.000000,0.000000,0.000000,-6.660000,-6.660000,-6.660000,1.000000,2019.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,2.800000,1.400000,2.100000,5.000000,2019.0
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,3.000000,15.000000,0.000000,0.000000,7.100000,4.400000,5.900000,9.000000,2019.0
75%,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,11.000000,0.000000,4.000000,5.000000,40.000000,0.000000,0.000000,13.600000,10.300000,12.000000,14.000000,2019.0
max,517.000000,5.000000,5.000000,68.000000,45.000000,32.000000,225.000000,4.000000,15.000000,20.000000,226.000000,4.000000,3.000000,53.700000,43.760000,46.700000,17.000000,2019.0


In [7]:
# Top 5 Players with the most Passing TD (all weeks combined)
top_5_assists_overall = df.groupby('Player')[['PassingTD']].sum()
top_5_assists_overall = top_5_assists_overall.sort_values(by='PassingTD', ascending=False).head(5)
top_5_assists_overall

,PassingTD
Player,
Lamar Jackson,36.0
Jameis Winston,33.0
Russell Wilson,31.0
Dak Prescott,30.0
Jimmy Garoppolo,27.0


In [8]:
# Plot - Top 5 Passers Overall 

top_5_assists_overall = pd.DataFrame(top_5_assists_overall)
source = ColumnDataSource(top_5_assists_overall)

hover = HoverTool(tooltips = [("Player", "@Player"), ("Passing TD", "@PassingTD")])

names_list = source.data['Player'].tolist()
p1 = figure(
       y_range=names_list,
       title='TOP 5 PASSERS',
       x_axis_label='Passing TouchDowns',
       y_axis_label='',
       plot_height=500,
       sizing_mode="stretch_width",
       tools=[hover,'save', 'reset', 'wheel_zoom']
)

p1.hbar(y='Player', 
       right='PassingTD', 
       height=0.4, 
       fill_color=factor_cmap(
              'Player',
              palette=Viridis[5],
              factors=names_list
       ),
       alpha=0.8,
       source=source
)

p1.title.text_font_size='28px'
p1.title.align='center'
p1.title.text_color='#2e3333'
p1.xgrid.grid_line_color=None

show(p1)

In [9]:
# Top 5 Players With Most Interceptions
top_5_interceptions = df.groupby('Player')[['Int']].sum()
top_5_interceptions = top_5_interceptions.sort_values(by='Int', ascending=False).head(5)
top_5_interceptions

,Int
Player,
Jameis Winston,30.0
Baker Mayfield,21.0
Philip Rivers,20.0
Jared Goff,16.0
Kyle Allen,16.0


In [10]:
# Plot - Top 5 Passers Overall

top_5_interceptions = pd.DataFrame(top_5_interceptions)
source = ColumnDataSource(top_5_interceptions)

hover = HoverTool(tooltips = [("Player", "@Player"), ("Interceptions", "@Int")])

names_list = source.data['Player'].tolist()
p2 = figure(
       y_range=names_list,
       title='TOP 5 INTERCEPTORS',
       x_axis_label='Interceptions',
       y_axis_label='',
       plot_height=500,
       sizing_mode="stretch_width",
       tools=[hover,'save', 'reset', 'wheel_zoom']
)

p2.hbar(y='Player', 
       right='Int', 
       height=0.4, 
       fill_color=factor_cmap(
              'Player',
              palette=Viridis[5],
              factors=names_list
       ),
       alpha=0.8,
       source=source
)

p2.title.text_font_size='28px'
p2.title.align='center'
p2.title.text_color='#2e3333'
p2.xgrid.grid_line_color=None

show(p2)

In [11]:
# Top 5 Players With Most Fumbles (all weeks combined)
top_5_fumbles = df.groupby('Player')[['FL']].sum()
top_5_fumbles = top_5_fumbles.sort_values(by='FL', ascending=False).head(5)
top_5_fumbles

,FL
Player,
Daniel Jones,11.0
Kyle Allen,7.0
Carson Wentz,7.0
Gardner Minshew,7.0
Jameis Winston,5.0


In [12]:
# Plot - Top 5 Fumbles

top_5_fumbles = pd.DataFrame(top_5_fumbles)
source = ColumnDataSource(top_5_fumbles)

hover = HoverTool(tooltips = [("Player", "@Player"), ("Fumbles", "@FL")])

names_list = source.data['Player'].tolist()
p3 = figure(
       y_range=names_list,
       title='TOP 5 FUMBLES',
       x_axis_label='Fumbles',
       y_axis_label='',
       plot_height=500,
       sizing_mode="stretch_width",
       tools=[hover,'save', 'reset', 'wheel_zoom']
)

p3.hbar(y='Player', 
       right='FL', 
       height=0.4, 
       fill_color=factor_cmap(
              'Player',
              palette=Viridis[5],
              factors=names_list
       ),
       alpha=0.8,
       source=source
)

p3.title.text_font_size='28px'
p3.title.align='center'
p3.title.text_color='#2e3333'
p3.xgrid.grid_line_color=None


show(p3)

In [13]:
# Creating Panel

tab1 = Panel(child=p1, title="Top 5 Passers")
tab2 = Panel(child=p2, title="Top 5  Interceptors")
tab3 = Panel(child=p3, title="Top 5 Fumbles")

show(Tabs(tabs=[tab1, tab2, tab3]))

In [14]:
# Teams ith the most pass completion 
teams_completion = df.groupby('Team')[['Cmp']].sum()
teams_completion = teams_completion.sort_values(by='Cmp', ascending=False)
teams_completion

,Cmp
Team,
ATL,459.0
NOR,418.0
LAR,397.0
LAC,394.0
PHI,391.0
DAL,388.0
CAR,382.0
TAM,382.0
NWE,378.0


In [16]:
# Plot - Teams Completion

teams_completion = pd.DataFrame(teams_completion)
source = ColumnDataSource(teams_completion)

hover = HoverTool(tooltips = [("Team", "@Team"), ("Completion", "@Cmp")])

teams_list = source.data['Team'].tolist()

p_teams = figure(
       x_range=teams_list,
       title='TEAMS COMPLETION',
       x_axis_label='Teams',
       y_axis_label='Completion',
       plot_height=500,
       plot_width=1200,
       tools=[hover,'save', 'reset', 'wheel_zoom']
)

s = p_teams.circle(
    x='Team', 
    y='Cmp',
    size=30, 
    fill_color=factor_cmap(
        'Team',
        palette=('#440154', '#482374', '#472777', '#472C7B', '#45347F', '#433C84', '#433C84', '#404387', '#3B508A', '#39558B', '#365A8C', '#3F4788','#345E8D', '#2C728E', '#267F8E', '#228B8D','#208F8C', '#29788E', '#277D8E', '#25838D', '#228B8D', '#22A784', '#1E988A', '#39B976', '#42BE71', '#79D151', '#81D34C', '#9FD938', '#BADE27', '#C5DF21', '#C7E01F','#FDE724'),
        factors=teams_list
       ),
    alpha=0.8,
    line_color=None,
    source=source)

p_teams.title.text_font_size='28px'
p_teams.title.align='center'
p_teams.title.text_color='#2e3333'

slider = Slider(start=0.25, end=1, step=0.01, value=0.25)
slider.js_link('value', s.glyph, 'radius')

show(column(p_teams, slider))